# Ejercicio 3: Plot the monthly number of searches 

##       for flights arriving at Málaga, Madrid or Barcelona

# Team Science t-ish

## Importamos librerias

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#Aeropuertos que queremos analizar: Madrid, Barcelona y Málaga
iata_codes = ["MAD", "BCN", "AGP"]

In [3]:
df_filter = pd.DataFrame()
chunksize = 999999 

reader =  pd.read_csv(
    'searches.csv.bz2', 
    compression = "bz2", 
    sep = "^", 
    memory_map = True, 
    error_bad_lines = False,
    usecols=["Date","Destination"],
    chunksize=chunksize
)

In [4]:
%%time
for df in reader:      
    curr = df[df["Destination"].isin(iata_codes)]
    if df_filter.empty:
        df_filter = curr
    else:
        df_filter = pd.concat([df_filter, curr])

CPU times: user 1min 53s, sys: 2.74 s, total: 1min 56s
Wall time: 1min 57s


In [ ]:
df_filter.head(2)

In [ ]:
df_filter.shape

In [ ]:
df_filter['Date']=pd.to_datetime(df_filter['Date'])
df_filter['Mes']=df_filter['Date'].dt.month

In [ ]:
#Agrupo por destino y mes
df_agrupado = df_filter.groupby(by=["Destination", "Mes"]).count()

In [ ]:
df_agrupado

In [ ]:
df_agrupado_1 = df_agrupado.reset_index()  #Paso los Indices a Columnas
df_agrupado_1.head(2)

In [ ]:
#Renombramos el título del campo que arrastamos de "Date" a "Cuenta"
df_agrupado_1 = df_agrupado_1.rename(columns={'Date': 'Cuenta'})
df_agrupado_1.head()

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))

#En eje X saco con .index todos los posibles valores de "Mes"
eje_x=df_agrupado_1['Mes'].value_counts().index

#En eje Y represento la cuenta total para cada mes y para cada Aeroupuerto

sns.lineplot(data=df_agrupado_1, x="Mes", y="Cuenta", hue="Destination", ax=ax)

In [ ]:
d={'Meses': ['Enero','Febrero','Marzo','Abril','Mayo','Junio','Julio',
             'Agosto','Septiembre','Octubre','Noviembre','Diciembre'],
  'Mes': [1,2,3,4,5,6,7,8,9,10,11,12]}
df_meses=pd.DataFrame(data=d)
df_meses

In [ ]:
#Hacemos un Merge entre la tabla de meses y la de agrupaciones
df_agrupado_1 = pd.merge(df_agrupado_1, df_meses, on="Mes", how='left')

In [ ]:
df_agrupado_1

## Rehacemos gráfica cambiando el eje x por los nombres de meses 

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))

#En eje X saco con .index todos los posibles valores de "Mes"
eje_x=df_agrupado_1['Mes'].value_counts().index

#En eje Y represento la cuenta total para cada mes y para cada Aeroupuerto

sns.lineplot(data=df_agrupado_1, x="Meses", y="Cuenta", hue="Destination", ax=ax)